Access Salesforce through Python based on OAuth method
Primary Source: https://jereze.com/code/authentification-salesforce-rest-api-python/

In [5]:
import json
import datetime
import requests
import base64

from simple_salesforce.api import SFType

In [6]:
with open("login_oauth.json", "r") as login_file:
    creds = json.load(login_file)

In [7]:
#Login 
PARAMS = creds
authorization_response = requests.post(url='https://login.salesforce.com/services/oauth2/token', data=PARAMS)
print('authorization response: {}'.format(authorization_response))
response=authorization_response.json()
access_token = response['access_token']
instance_url = response['instance_url']

authorization response: <Response [200]>


In [8]:
def sf_api_call(action, parameters = {}, method = 'get', data = {}):
    """
    Helper function to make calls to Salesforce REST API.
    Parameters: action (the URL), URL params, method (get, post or patch), data for POST/PATCH.
    """
    headers = {
        'Content-type': 'application/json',
        'Accept-Encoding': 'charset=UTF-8',
        'Authorization': 'Bearer %s' % access_token
    }
    print (access_token)
    print (instance_url)
    if method == 'get':
        r = requests.request(method, instance_url+action, headers=headers, params=parameters, timeout=30)
    elif method in ['post', 'patch']:
        r = requests.request(method, instance_url+action, headers=headers, json=data, params=parameters, timeout=10)
    else:
        # other methods not implemented in this example
        raise ValueError('Method should be get or post or patch.')
    print('Debug: API %s call: %s' % (method, r.url) )
    if r.status_code < 300:
        if method=='patch':
            return None
        else:
            return r.json()
    else:
        raise Exception('API error when calling %s : %s' % (r.url, r.content))


In [ ]:

full_aava_code='AU08LQGD4A3XD06.1'
print(json.dumps(sf_api_call('/services/data/v42.0/query/', {
    'q': "SELECT Id FROM Product2 Where SalesProductCodewithVersion__c = 'AU08LQGD4A3XD06.1'"
}), indent=2))


In [ ]:
full_aava_code='AU08LQGD4A3XD06.1'
test=sf_api_call('/services/data/v42.0/query/', {
    'q': "SELECT Id FROM Product2 Where SalesProductCodewithVersion__c = '" + full_aava_code + "'"
})

In [11]:
test=test['records'][0]

In [ ]:
test['Id']

In [ ]:
#upload data, needs to be in dictionary
for i in json_dicti:
    call = sf_api_call('/services/data/v42.0/sobjects/Shipping__c/', method="post", data=i)
    shipping_id = call.get('id')